In [1]:
import pandas as pd
import glob
import decimal
import pyodbc

In [2]:
conn_prod = pyodbc.connect('Driver={SQL Server}; Server=10.241.109.41,20010\\OASYSHDB;Database=EPSA_Reporting;UID=epsareportes; PWD=Epsa.2020!;')

In [3]:
dff=pd.read_sql_query("""EXEC	[EPSA_Reporting].[dbo].[sp_AgregacionAnalogaHora] 	@f_ini = N'2022-06-18 00:00:00', @f_fin = N'2022-06-20 00:00:00', @Measure_Type = N'POTENCIA ACTIVA', @p_Unit = N'MW' """,
conn_prod,
coerce_float=False
).fillna(0)

##dff1=pd.read_sql_query("""EXEC	[EPSA_Reporting].[dbo].[sp_AgregacionAnalogaHora] 	@f_ini = N'2021-07-01 00:00:00', @f_fin = N'2021-07-02 00:00:00', @Measure_Type = N'POTENCIA ACTIVA', @p_Unit = N'MW' """,
##conn_prod,
##coerce_float=False
##).fillna(0)

##dff2=pd.read_sql_query("""EXEC	[EPSA_Reporting].[dbo].[sp_AgregacionAnalogaHora] 	@f_ini = N'2021-08-01 00:00:00', @f_fin = N'2021-08-02 00:00:00', @Measure_Type = N'POTENCIA ACTIVA', @p_Unit = N'MW' """,
##conn_prod,
##coerce_float=False
##).fillna(0)

##dff3=pd.read_sql_query("""EXEC	[EPSA_Reporting].[dbo].[sp_AgregacionAnalogaHora] 	@f_ini = N'2021-09-01 00:00:00', @f_fin = N'2021-09-02 00:00:00', @Measure_Type = N'POTENCIA ACTIVA', @p_Unit = N'MW' """,
##conn_prod,
##coerce_float=False
##).fillna(0)

##dff4=pd.read_sql_query("""EXEC	[EPSA_Reporting].[dbo].[sp_AgregacionAnalogaHora] 	@f_ini = N'2021-10-01 00:00:00', @f_fin = N'2021-10-02 00:00:00', @Measure_Type = N'POTENCIA ACTIVA', @p_Unit = N'MW' """,
##conn_prod,
##coerce_float=False
##).fillna(0)

C:\Users\mfhernandezp\AppData\Roaming\Python\Python310\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [4]:
#df=pd.concat([dff,dff1,dff2,dff3,dff4],sort=False)
#df=dff.values.tolist()
df=dff
#print (df)

In [5]:
df['Hour'] = df['FECHA_FIN'].str.split(' ', expand=True)[1]



In [6]:
import datetime
from datetime import datetime
df['date'] = df['FECHA_FIN'].str.split(' ', expand=True)[0]
df['Day']= pd.to_datetime(df['date']).dt.day_name()
PromH=df.groupby(['PUNTO_SCADA', 'Day', 'Hour']).AVG.mean()#.reset_index(drop=False)
PromD=df.groupby(['PUNTO_SCADA', 'Day']).AVG.mean()#.reset_index(drop=False)
df2=pd.merge(PromH,PromD,on='PUNTO_SCADA').reset_index(drop=False)
df["PU"]= df2["AVG_x"].div(df2["AVG_y"].values).fillna(0)
df
print(df['date'])

0         2022-06-18
1         2022-06-18
2         2022-06-18
3         2022-06-18
4         2022-06-18
             ...    
155803    2022-06-20
155804    2022-06-20
155805    2022-06-20
155806    2022-06-20
155807    2022-06-20
Name: date, Length: 155808, dtype: object


In [7]:
import holidays_co
hol = pd.DataFrame(holidays_co.get_colombia_holidays_by_year(2020))
hol1= pd.DataFrame(holidays_co.get_colombia_holidays_by_year(2021))
hol2=pd.DataFrame(holidays_co.get_colombia_holidays_by_year(2022))
holiday=pd.concat([hol,hol1,hol2],axis=0, sort=False, ignore_index=True)
print(holiday['date'])


0     2020-01-01
1     2020-01-06
2     2020-03-23
3     2020-04-09
4     2020-04-10
5     2020-05-01
6     2020-05-25
7     2020-06-15
8     2020-06-22
9     2020-06-29
10    2020-07-20
11    2020-08-07
12    2020-08-17
13    2020-10-12
14    2020-11-02
15    2020-11-16
16    2020-12-08
17    2020-12-25
18    2021-01-01
19    2021-01-11
20    2021-03-22
21    2021-04-01
22    2021-04-02
23    2021-05-01
24    2021-05-17
25    2021-06-07
26    2021-06-14
27    2021-07-05
28    2021-07-20
29    2021-08-07
30    2021-08-16
31    2021-10-18
32    2021-11-01
33    2021-11-15
34    2021-12-08
35    2021-12-25
36    2022-01-01
37    2022-01-10
38    2022-03-21
39    2022-04-14
40    2022-04-15
41    2022-05-01
42    2022-05-30
43    2022-06-20
44    2022-06-27
45    2022-07-04
46    2022-07-20
47    2022-08-07
48    2022-08-15
49    2022-10-17
50    2022-11-07
51    2022-11-14
52    2022-12-08
53    2022-12-25
Name: date, dtype: object


In [ ]:
for x in df['date']:
      for y in holiday['date']:
            if y in x:
                  df['Festivo']='1'
            else:
                  df['Festivo']='0'
                  


import csv
df.to_csv('Datos.csv', sep=";", quoting=csv.QUOTE_NONE, escapechar=" ")


In [ ]:
holiday

In [ ]:
['Yes' for i in df.date if i in holiday.date]

In [8]:
for i in df.date:
    if i in holiday.date:
        print(i)

In [9]:
holiday.date[1]

datetime.date(2020, 1, 6)

In [10]:
df.date[0]

'2022-06-18'